In [1]:
import sys

import tensorflow as tf
from tensorflow import keras as kr

from time import sleep
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_rows', 200)

import sklearn as sk

from sklearn.preprocessing import MinMaxScaler

import matplotlib 

import matplotlib.pyplot as plt

import zipfile
import os

import supportFunctions as sf

import inspect
lines = inspect.getsource(sf.highest)

In [2]:
#função utilizada para normalizar os dados de treino e validação
def normalize_data(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    
    return scaler, train_scaled, test_scaled

#função utilizada para desnormalizar os dados, no formato de entrada
def denormalize_data(scaler,data):
    result = scaler.inverse_transform(data)
    return result

#função utilizada para desnomalizar o valor predito
def denormalize_prediction(scaler, dataX, dataY):
    formatted_data = np.array(dataX)
    formatted_data[:,3] = np.array(dataY)
    
    return denormalize_data(scaler,formatted_data)[:,3]

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
# tf.debugging.set_log_device_placement(False)

Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
Pandas 1.5.2
Scikit-Learn 1.2.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [4]:
#quantos passos para trás os indicadores técnicos vão olhar
ti_memory = 10

#lendo os dados que serão utilizados para treinamento e validação
ge1day = pd.read_csv('Data\GE.csv')
#removendo a coluna que indica a data
ge1day = ge1day.drop(['Date'], axis = 1)

df = ge1day
#reseting the index
df.index = range(len(df))

In [7]:
#adicionando os parametros de indicadores tecnicos ao dataframe
df["""highest_{}""".format(ti_memory)] = sf.highest(df.Close,ti_memory)
df["""lowest_{}""".format(ti_memory)] = sf.lowest(df.Close,ti_memory)
df["""wma_{}""".format(ti_memory)] = sf.wma(df.Close,ti_memory)
df["""ema_{}""".format(ti_memory)] = sf.ema(df.Close,ti_memory)
df["""hma_{}""".format(ti_memory)] = sf.hma(df.Close,ti_memory)
df["""macd_12_26"""] = sf.macd(df.Close,12,26)
df["""rsi_{}""".format(ti_memory)] = sf.rsi(df.Close,ti_memory)
df["""dpo_{}""".format(ti_memory)] = sf.dpo(df.Close,ti_memory)

#parametros nao utilizados:
# df["""sma_{}""".format(ti_memory)] = sf.sma(df.Close,ti_memory)
# df["""so_k_5"""] = sf.so_k(df.Close)
# df["""so_d_3"""] = sf.so_d(df.Close)
# df["""obv""".format(ti_memory)] = sf.obv(df.Close,df.Volume)

C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: divide by zero encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division
C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: invalid value encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division


In [8]:
display(df)

,Open,High,Low,Close,Adj Close,Volume,highest_10,lowest_10,wma_10,ema_10,hma_10,macd_12_26,rsi_10,dpo_10
0,0.751202,0.763722,0.743690,0.748698,0.001782,2156500,0.748698,0.748698,0.748698,0.748698,0.748698,0.000000,NaN,0.000000
1,0.744942,0.744942,0.738682,0.741186,0.001764,1477600,0.748698,0.741186,0.743690,0.744942,0.745359,0.000000,100.000000,-0.744942
2,0.741186,0.747446,0.726162,0.732422,0.001743,1837000,0.748698,0.732422,0.738056,0.740769,0.741708,0.000000,100.000000,-0.740769
3,0.732422,0.733674,0.701122,0.713642,0.001698,2725600,0.748698,0.713642,0.728290,0.733987,0.734112,0.000000,100.000000,-0.733987
4,0.713642,0.713642,0.691106,0.712390,0.001695,3095000,0.748698,0.712390,0.722990,0.729667,0.727268,0.000000,100.000000,-0.729667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,7.630000,8.300000,7.510000,8.120000,8.120000,123180900,8.120000,6.110000,7.062182,7.424700,7.239323,-1.116727,50.108961,-0.399000
14659,7.680000,7.870000,7.540000,7.620000,7.620000,93299000,8.120000,6.110000,7.175091,7.460209,7.697586,-1.029444,40.027642,-0.496000
14660,7.540000,7.940000,7.350000,7.890000,7.890000,86850200,8.120000,6.110000,7.341273,7.538353,8.019525,-0.927790,41.907705,-0.579000
14661,7.870000,8.180000,7.820000,7.940000,7.940000,121149900,8.120000,6.110000,7.494182,7.611380,8.165303,-0.833585,32.316173,-1.075000


In [9]:
#jogando alguns dados fora para termos todas variaveis incializadas
inicialization_steps = 30
prediction_ahead = 1

dataValues = df.values

trainSize = int(len(dataValues)*0.8)
testSize = len(dataValues) - trainSize

train = df.head(trainSize)
# train,minDataTrain,minMaxDataTrain = sf.normalizaTrain(train)
test = df.tail(testSize)
# test = sf.normalizaTest(test, minDataTrain, minMaxDataTrain)

scaler, train_normalized, test_normalized = normalize_data(train, test)

#deixando um valor de fora, para podermos prever o próximo valor quando for o último:
trainX = train_normalized[inicialization_steps:(len(train)-prediction_ahead)]
testX = test_normalized[0:(len(test)-prediction_ahead)]

#normalizando as entradas
# scalerX, trainX, testX = normalize_data(trainX, testX)

#pegando apenas o valor que queremos prever
trainY = train_normalized[(inicialization_steps+prediction_ahead):len(train),3]
testY = test_normalized[prediction_ahead:len(test),3]

#calculando a diferenca percentual do preco de fechamento entre um dia e outro
# trainY = (train.values[(inicialization_steps+1):len(train),3]/trainX[:,3])-1
# testY = (test.values[1:len(test),3]/testX[:,3])-1


# scalerY, trainY, testY = normalize_data(trainY, testY)

In [10]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(11699, 14)
(11699,)
(2932, 14)
(2932,)


In [11]:
#checking if it makes sense
# print(trainInputX[:10,(rnn_memory-1),3])
# print(trainInputY[:10])

In [12]:
trainInputX = trainX
trainInputY = trainY
testInputX = testX
testInputY = testY
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,0:14]
testInputX = testInputX [:,0:14]
print(trainInputX.shape)

trained_models_MLP = []
trained_models_MLP_history = []

for i in range(0,60):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 64

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_MLP.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_MLP[i].add(kr.layers.Dense(14))
    trained_models_MLP[i].add(kr.layers.Dense(32))
    trained_models_MLP[i].add(kr.layers.Dense(32))
    trained_models_MLP[i].add(kr.layers.Dense(32))
    trained_models_MLP[i].add(kr.layers.Dense(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_MLP[i].add(kr.layers.Dense(1))
    trained_models_MLP[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_MLP_history.append(trained_models_MLP[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_MLP[i].save('C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_'+str(i))

(11699, 14)
(11699,)
(2932, 14)
(2932,)
11699
11699
11699
11699
(11699, 14)
Epoch 1/2000
183/183 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 1.3060e-04
Epoch 2/2000
183/183 [==============================] - 0s 737us/step - loss: 9.6410e-05 - val_loss: 7.1057e-05
Epoch 3/2000
183/183 [==============================] - 0s 748us/step - loss: 7.4566e-05 - val_loss: 5.8033e-05
Epoch 4/2000
183/183 [==============================] - 0s 742us/step - loss: 7.5127e-05 - val_loss: 8.9520e-05
Epoch 5/2000
183/183 [==============================] - 0s 731us/step - loss: 7.5438e-05 - val_loss: 1.3132e-04
Epoch 6/2000
183/183 [==============================] - 0s 742us/step - loss: 6.3502e-05 - val_loss: 5.6957e-05
Epoch 7/2000
183/183 [==============================] - 0s 748us/step - loss: 6.4907e-05 - val_loss: 1.4351e-04
Epoch 8/2000
183/183 [==============================] - 0s 748us/step - loss: 7.0258e-05 - val_loss: 1.1987e-04
Epoch 9/2000
183/183 [============

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
183/183 [==============================] - 1s 1ms/step - loss: 0.0124 - val_loss: 0.0043
Epoch 2/2000
183/183 [==============================] - 0s 731us/step - loss: 1.6778e-04 - val_loss: 9.0531e-04
Epoch 3/2000
183/183 [==============================] - 0s 759us/step - loss: 1.1185e-04 - val_loss: 3.2231e-04
Epoch 4/2000
183/183 [==============================] - 0s 759us/step - loss: 8.4759e-05 - val_loss: 2.1321e-04
Epoch 5/2000
183/183 [==============================] - 0s 748us/step - loss: 7.2439e-05 - val_loss: 6.7189e-05
Epoch 6/2000
183/183 [==============================] - 0s 742us/step - loss: 6.2655e-05 - val_loss: 5.5433e-05
Epoch 7/2000
183/183 [==============================] - 0s 748us/step - loss: 6.1872e-05 - val_loss: 5.4676e-05
Epoch 8/2000
183/183 [==============================] - 0s 770us/step - loss: 7.0727e-05 - val_loss: 7.0186e-05
Epoch 9/2000
183/183 [==============================] - 0s 709us/step - loss: 5.8293e-05 - val_loss: 5.2153e-05
Ep

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_58\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_58\assets


Epoch 1/2000
183/183 [==============================] - 1s 1ms/step - loss: 0.0047 - val_loss: 1.9869e-04
Epoch 2/2000
183/183 [==============================] - 0s 731us/step - loss: 7.2052e-05 - val_loss: 8.4082e-05
Epoch 3/2000
183/183 [==============================] - 0s 731us/step - loss: 6.1729e-05 - val_loss: 6.3352e-05
Epoch 4/2000
183/183 [==============================] - 0s 731us/step - loss: 6.2486e-05 - val_loss: 6.2067e-05
Epoch 5/2000
183/183 [==============================] - 0s 720us/step - loss: 5.8913e-05 - val_loss: 6.0648e-05
Epoch 6/2000
183/183 [==============================] - 0s 726us/step - loss: 5.5128e-05 - val_loss: 6.4410e-05
Epoch 7/2000
183/183 [==============================] - 0s 731us/step - loss: 5.6801e-05 - val_loss: 6.5814e-05
Epoch 8/2000
183/183 [==============================] - 0s 720us/step - loss: 5.4490e-05 - val_loss: 5.9483e-05
Epoch 9/2000
183/183 [==============================] - 0s 715us/step - loss: 5.6889e-05 - val_loss: 9.9104e-0

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_59\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/MLP_predicting1ahead_param_14_32_32_32_8_batch_256_earlystop_valloss_59\assets
